In [ ]:
import time

notebook_start_time = time.time()

# Setup Environment

In [ ]:
import sys
from pathlib import Path
from IPython import get_ipython

In [ ]:
def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

In [ ]:
def clone_repository() -> None:
    !git clone https://github.com/haturusinghe/mlops-datascience-anime-recsys.git
    %cd mlops-datascience-anime-recsys/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !!uv pip install --all-extras --system --prerelease disallow --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

In [ ]:
import os

if os.getcwd() != '/content/mlops-datascience-anime-recsys':
    %cd mlops-datascience-anime-recsys

In [ ]:
if is_google_colab():
    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

# Feature Pipeline

## Imports

In [ ]:
# %load_ext autoreload
# %autoreload 2

import warnings
from pprint import pprint

import polars as pl
import torch
from loguru import logger
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")

from recsys.raw_data_sources import myanimelist_dataset
from recsys.features.anime import (compute_features_of_anime, generate_embeddings_for_dataframe)
from recsys.features.user import (compute_features_of_user)
from recsys.config import settings

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [ ]:
print(dict(settings))

## MyAnimeList Dataset

### Anime Data

- MAL_ID - MyAnimelist ID of the anime.

In [ ]:
anime_df = myanimelist_dataset.extract_anime_data()
anime_df.shape

In [ ]:
anime_df.head(3)

In [ ]:
anime_df = compute_features_of_anime(anime_df)
anime_df.shape


#### Create an embedding for each anime description

In [ ]:
for i, desc in enumerate(anime_df["description"].head(n=3)):
    logger.info(f"Item {i+1}:\n{desc}")

In [ ]:
logger.info(
    f"Loading '{settings.FEATURES_EMBEDDING_MODEL_ID}' embedding model to {device=}"
)

In [ ]:
# Load the embedding model from SentenceTransformer's model registry.
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

In [ ]:
anime_df = generate_embeddings_for_dataframe(
    anime_df,
    model=model,
    embedding_column_name="description",
    batch_size=128,
)

In [ ]:
anime_df[["article_description", "embeddings"]].head(3)

## User Data

In [ ]:
users_df = myanimelist_dataset.extract_user_data()
users_df.shape

In [ ]:
users_df.head(3)

In [ ]:
users_df.null_count()

In [ ]:
users_df = compute_features_of_user(users_df)